# Optical Lever Sensing Matrices

In this tutorial, we will demonstrate the use of `kontrol.OpticalLeverSensingMatrix` class, which inherit `kontrol.SensingMatrix`. `kontrol.OpticalLeverSensingMatrix` is a general sensing matrix for optical levers with optical lever beams that has tilted incidence plane with respect to the horizontal/vertical plane. Realistically, we won't be needing this general matrix since optical levers (in KAGRA) are roughly horizontal or vertical. So, we provide reduced version of `kontrol.OpticalLeverSensingMatrix`, namely, `kontrol.HorizontalOpticalLeverSensingMatrix` and `kontrol.VerticalOpticalLeverSensingMatrix` classes, which are more useful practically.

